In [1]:
#%matplotlib widget
from backtrader import plot
import backtrader as bt
import backtrader.analyzers as btanalyzers
import rba_tools as rba
import pandas as pd
import numpy as np
#import matplotlib
from datetime import datetime,date,timedelta
from backtrader.utils import num2date,date2num
import rba_tools.backtest.rba_backtrader_set as rbs
import plotly.graph_objects as go
import plotly.express as px
import rba_tools.backtest.backtrader_extensions.plot as rbsplot
import importlib


In [2]:
importlib.reload(rbsplot)

<module 'rba_tools.backtest.backtrader_extensions.plot' from 'c:\\users\\avery\\documents\\github\\rba_tools_clone\\src\\rba_tools\\backtest\\backtrader_extensions\\plot.py'>

In [3]:
class MaCrossStrategy(bt.Strategy):

    def __init__(self):
        ma_fast = bt.ind.SMA(period = 10)
        ma_slow = bt.ind.SMA(period = 50)

        self.crossover = bt.ind.CrossOver(ma_fast, ma_slow)

    def next(self):
        if not self.position:
            if self.crossover > 0:
                self.buy()
        elif self.crossover < 0:
            self.close()

In [4]:
cerebro = bt.Cerebro()
puller = rba.retriever.get_crypto_data.DataPuller.kraken_puller()
symbol = 'ETH/USD'
from_date = '1/1/20'
to_date = '12/31/20'

In [5]:
dataframe = puller.fetch_df(symbol, 'h', from_date, to_date)
data = bt.feeds.PandasData(dataname=dataframe,
                               # datetime='Date',
                               nocase=True,
                               )
cerebro.adddata(data)
cerebro.addstrategy(MaCrossStrategy)
cerebro.broker.setcash(1000.0)
cerebro.addsizer(bt.sizers.PercentSizer, percents = 10)

In [6]:
back = cerebro.run()

In [7]:
strategy = back[0]
from backtrader import plot
plotter = plot.Plot()
plotter

In [8]:
plotter.sortdataindicators(strategy)

In [9]:
vars(plotter)

{'params': <backtrader.metabase.AutoInfoClass_Plot_OldSync at 0x1ba7b8cef10>,
 'p': <backtrader.metabase.AutoInfoClass_Plot_OldSync at 0x1ba7b8cef10>,
 'dplotstop': [<backtrader.observers.broker.Broker at 0x1ba7d3c4fa0>,
 'dplotsup': defaultdict(list, {}),
 'dplotsdown': defaultdict(list,
             {<backtrader.indicators.sma.SMA at 0x1ba7b8551f0>: [<backtrader.indicators.crossover.CrossOver at 0x1ba7b89beb0>]}),
 'dplotsover': defaultdict(list,
             {<backtrader.feeds.pandafeed.PandasData at 0x1ba7b87c5b0>: [<backtrader.observers.buysell.BuySell at 0x1ba7d3c4220>,
               <backtrader.indicators.sma.SMA at 0x1ba7b855af0>]})}

In [10]:
plot_result = plotter.plot(strategy)
plot_result

[<Figure size 640x480 with 5 Axes>]

In [11]:
st_dtime = strategy.lines.datetime.plot()

In [12]:
print(plotter)

In [13]:
vars(plotter)

{'params': <backtrader.metabase.AutoInfoClass_Plot_OldSync at 0x1ba7b8cef10>,
 'p': <backtrader.metabase.AutoInfoClass_Plot_OldSync at 0x1ba7b8cef10>,
 'dplotstop': [<backtrader.observers.broker.Broker at 0x1ba7d3c4fa0>,
 'dplotsup': defaultdict(list,
             {<backtrader.feeds.pandafeed.PandasData at 0x1ba7b87c5b0>: [],
              <backtrader.observers.buysell.BuySell at 0x1ba7d3c4220>: [],
              <backtrader.indicators.sma.SMA at 0x1ba7b8551f0>: [],
              <backtrader.indicators.sma.SMA at 0x1ba7b855af0>: [],
              <backtrader.indicators.crossover.CrossOver at 0x1ba7b89beb0>: []}),
 'dplotsdown': defaultdict(list,
             {<backtrader.indicators.sma.SMA at 0x1ba7b8551f0>: [<backtrader.indicators.crossover.CrossOver at 0x1ba7b89beb0>],
              <backtrader.observers.buysell.BuySell at 0x1ba7d3c4220>: [],
              <backtrader.indicators.crossover.CrossOver at 0x1ba7b89beb0>: [],
              <backtrader.indicators.sma.SMA at 0x1ba7b855af0>:

In [14]:
over = plotter.dplotsover
over

defaultdict(list,
            {<backtrader.feeds.pandafeed.PandasData at 0x1ba7b87c5b0>: [<backtrader.observers.buysell.BuySell at 0x1ba7d3c4220>,
             <backtrader.observers.broker.Broker at 0x1ba7d3c4fa0>: [],
             <backtrader.observers.trades.Trades at 0x1ba7d3c4490>: [],
             <backtrader.observers.buysell.BuySell at 0x1ba7d3c4220>: [],
             <backtrader.indicators.sma.SMA at 0x1ba7b8551f0>: [],
             <backtrader.indicators.sma.SMA at 0x1ba7b855af0>: [],
             <backtrader.indicators.crossover.CrossOver at 0x1ba7b89beb0>: []})

In [15]:
type(plotter.dplotsup)

collections.defaultdict

In [16]:
mylist = [i for i in plotter.dplotsover]
mylist

In [17]:
buysell = mylist[3]
buysell

In [18]:
ind = mylist[4]
indicator = ind

In [19]:
lineidx = 0
line = ind.lines[lineidx]
linealias = ind.lines._getlinealias(lineidx)

In [20]:
line

In [21]:
vals = line.plotrange(0,len(line))

In [22]:
np.arange(5)

array([0, 1, 2, 3, 4])

In [23]:
np.array(vals)

array([    nan,     nan,     nan, ..., 736.722, 737.312, 737.685])

In [24]:
fig = go.Figure(data=go.Scatter(x=np.arange(len(line)),y=np.array(vals)))
fig.show()

In [25]:
ser = pd.Series(strategy.datetime.plot())
x_datetime = np.array(ser.map(num2date))

In [26]:
myp = rbsplot.testplotind(ind, pd.to_datetime(x_datetime))
go.Figure(myp).show()

In [27]:
type(x_datetime)

numpy.ndarray

In [28]:
t = pd.to_datetime(x_datetime)
t

DatetimeIndex(['2020-01-01 00:00:00', '2020-01-01 01:00:00',
               '2020-01-01 02:00:00', '2020-01-01 03:00:00',
               '2020-01-01 04:00:00', '2020-01-01 05:00:00',
               '2020-01-01 06:00:00', '2020-01-01 07:00:00',
               '2020-01-01 08:00:00', '2020-01-01 09:00:00',
               ...
               '2020-12-31 14:00:00', '2020-12-31 15:00:00',
               '2020-12-31 16:00:00', '2020-12-31 17:00:00',
               '2020-12-31 18:00:00', '2020-12-31 19:00:00',
               '2020-12-31 20:00:00', '2020-12-31 21:00:00',
               '2020-12-31 22:00:00', '2020-12-31 23:00:00'],
              dtype='datetime64[ns]', length=8782, freq=None)

In [29]:
inf0 = rbsplot.get_line_plot_info(buysell, 0)
inf1 = rbsplot.get_line_plot_info(buysell, 1)
inf0

In [30]:
inf0._getkwargs(skip_=True)

OrderedDict([('marker', '^'),
             ('markersize', 8.0),
             ('color', 'lime'),
             ('fillstyle', 'full'),
             ('ls', '')])

In [31]:
inf1._getkwargs(skip_=True)

OrderedDict([('marker', 'v'),
             ('markersize', 8.0),
             ('color', 'red'),
             ('fillstyle', 'full'),
             ('ls', '')])

In [32]:
x = rbsplot.get_datetime(strategy)
f = rbsplot.testplotind(buysell, x)
go.Figure(f).show()

In [33]:
for line in buysell.lines:
    print(line)

In [34]:
isinstance(back[0], bt.Strategy)

True

In [35]:
df = rba.backtest.rba_backtrader_set.get_ohlcv_data_from_cerebro_run(back)
df

,Open,High,Low,Close,Volume
2020-01-01 00:00:00,128.66,128.66,128.19,128.42,515.780816
2020-01-01 01:00:00,128.33,130.05,128.30,130.05,555.054287
2020-01-01 02:00:00,130.20,130.58,130.05,130.44,582.304327
2020-01-01 03:00:00,130.48,130.48,129.61,129.63,480.119100
2020-01-01 04:00:00,129.73,130.13,129.67,129.67,239.555071
...,...,...,...,...,...
2020-12-31 19:00:00,740.83,742.97,738.44,739.53,1559.446486
2020-12-31 20:00:00,739.46,745.37,739.07,745.09,1365.984380
2020-12-31 21:00:00,745.38,745.68,737.49,739.08,1334.269917
2020-12-31 22:00:00,739.27,743.00,736.68,741.50,1388.991292


In [36]:
type(df['Open'])

pandas.core.series.Series

In [37]:
rbsplot.add_indicator_to_df(df, buysell, inplace=True)
df

,Open,High,Low,Close,Volume,buy (barplot=True bardist=0.015),sell (barplot=True bardist=0.015)
2020-01-01 00:00:00,128.66,128.66,128.19,128.42,515.780816,NaN,NaN
2020-01-01 01:00:00,128.33,130.05,128.30,130.05,555.054287,NaN,NaN
2020-01-01 02:00:00,130.20,130.58,130.05,130.44,582.304327,NaN,NaN
2020-01-01 03:00:00,130.48,130.48,129.61,129.63,480.119100,NaN,NaN
2020-01-01 04:00:00,129.73,130.13,129.67,129.67,239.555071,NaN,NaN
...,...,...,...,...,...,...,...
2020-12-31 19:00:00,740.83,742.97,738.44,739.53,1559.446486,NaN,NaN
2020-12-31 20:00:00,739.46,745.37,739.07,745.09,1365.984380,NaN,NaN
2020-12-31 21:00:00,745.38,745.68,737.49,739.08,1334.269917,NaN,756.8652
2020-12-31 22:00:00,739.27,743.00,736.68,741.50,1388.991292,NaN,NaN


In [39]:
for line_index in range(buysell.size()):
    line = buysell.lines[line_index]
    alias = buysell.lines._getlinealias(line_index)
    print(line_index, alias)
    indicator_vals = line.plotrange(0, len(line))

0 buy
1 sell


In [40]:
df

,Open,High,Low,Close,Volume,buy (barplot=True bardist=0.015),sell (barplot=True bardist=0.015)
2020-01-01 00:00:00,128.66,128.66,128.19,128.42,515.780816,NaN,NaN
2020-01-01 01:00:00,128.33,130.05,128.30,130.05,555.054287,NaN,NaN
2020-01-01 02:00:00,130.20,130.58,130.05,130.44,582.304327,NaN,NaN
2020-01-01 03:00:00,130.48,130.48,129.61,129.63,480.119100,NaN,NaN
2020-01-01 04:00:00,129.73,130.13,129.67,129.67,239.555071,NaN,NaN
...,...,...,...,...,...,...,...
2020-12-31 19:00:00,740.83,742.97,738.44,739.53,1559.446486,NaN,NaN
2020-12-31 20:00:00,739.46,745.37,739.07,745.09,1365.984380,NaN,NaN
2020-12-31 21:00:00,745.38,745.68,737.49,739.08,1334.269917,NaN,756.8652
2020-12-31 22:00:00,739.27,743.00,736.68,741.50,1388.991292,NaN,NaN


In [42]:
rbsplot.add_indicator_to_df(df, ind, inplace=True)
df

,Open,High,Low,Close,Volume,buy (barplot=True bardist=0.015),sell (barplot=True bardist=0.015),sma (period=10)
2020-01-01 00:00:00,128.66,128.66,128.19,128.42,515.780816,NaN,NaN,NaN
2020-01-01 01:00:00,128.33,130.05,128.30,130.05,555.054287,NaN,NaN,NaN
2020-01-01 02:00:00,130.20,130.58,130.05,130.44,582.304327,NaN,NaN,NaN
2020-01-01 03:00:00,130.48,130.48,129.61,129.63,480.119100,NaN,NaN,NaN
2020-01-01 04:00:00,129.73,130.13,129.67,129.67,239.555071,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2020-12-31 19:00:00,740.83,742.97,738.44,739.53,1559.446486,NaN,NaN,738.091
2020-12-31 20:00:00,739.46,745.37,739.07,745.09,1365.984380,NaN,NaN,737.350
2020-12-31 21:00:00,745.38,745.68,737.49,739.08,1334.269917,NaN,756.8652,736.722
2020-12-31 22:00:00,739.27,743.00,736.68,741.50,1388.991292,NaN,NaN,737.312
